# **Data Merging Notebook**
This notebook includes code for merging different data.
The steps involved are:
1. Importing the data from respective files 
2. Check differences between Kenya and Nigeria data
3. Correct differences
4. Merge on common columns

In [26]:
# import required libraries
import pandas as pd

In [27]:
# read all the data into dataframes
ken_cust_df = pd.read_csv("../data/Kenya Customers.csv") # Kenya customer data
ken_del_df = pd.read_csv("../data/Kenya Deliveries.csv") # Kenya deliveries data
ken_ord_df = pd.read_csv("../data/Kenya Orders.csv") # Kenya orders data
nig_cust_df = pd.read_csv("../data/Nigeria Customers.csv") # Nigeria customer data
nig_del_df = pd.read_csv("../data/Nigeria Deliveries.csv") # Nigeria deliveries data
nig_ord_df = pd.read_csv("../data/Nigeria Orders.csv") # Nigeria orders data

### Identify common columns

In [28]:
# Check if the correspnding datasets in Nigeria and Kenya are the same
print("Customer Data")
print(set(ken_cust_df.columns) ^ set(nig_cust_df.columns))
print("Deliveries Data")
print(set(ken_del_df.columns) ^ set(nig_del_df.columns))
print("Order Data")
print(set(ken_ord_df.columns) ^ set(nig_ord_df.columns))


Customer Data
{'Number of Employees', 'Upload restuarant location', 'Number of employees'}
Deliveries Data
{'Unnamed: 35', 'Unnamed: 34', 'Agent_Name'}
Order Data
{'Debt Amount'}


### Differences between the Nigeria and Kenya datasets
1. The customer data in Kenya dataset has "Upload restuarant location" and Nigeria dataset does not. 
2. 'Number of Employees' in the customer data is spelt different in Kenya and Nigeria datasets.
3. The deliveries data in Kenya dataset has 'Unnamed: 34', 'Unnamed: 35', 'Agent_Name' and Nigeria dataset does not.
4. The orders data in Kenya Dataset has 'Debt Amount' and Nigeria dataset does not.

### Correct differences

In [29]:
# Spell 'Number of Employees' in the customer data the same.
ken_cust_df = ken_cust_df.rename(columns={'Number of employees':'Number of Employees'})

### Concatenate different countries data

In [30]:
# Add country codes 
nig_cust_df["country_code"]="234"
nig_del_df["country_code"]="234"
nig_ord_df["country_code"]="234"

ken_cust_df["country_code"]="254"
ken_del_df["country_code"]="254"
ken_ord_df["country_code"]="254"

# Append datasets by countries
cust_df = ken_cust_df.append(nig_cust_df)
del_df = ken_del_df.append(nig_del_df)
ord_df = ken_ord_df.append(nig_ord_df)

### Check for common columns among customer, deliveries and order data

In [31]:
# Check if there are common columns among customer, deliveries and order data in Kenya
print("Customer and Deliveries Data")
print(set(cust_df.columns) & set(del_df.columns))
print("Deliveries and Orders Data")
print(set(del_df.columns) & set(ord_df.columns))
print("Customer and Order Data")
print(set(ord_df.columns) & set(cust_df.columns))

Customer and Deliveries Data
{'country_code'}
Deliveries and Orders Data
{'country_code', 'Tip', 'Discount'}
Customer and Order Data
{'country_code', 'Customer ID'}


### Identified common columns
1. Deliveries and Orders Data have {'Discount', 'Tip'} in common but they are not unique ID type columns. Delivery data has "Order_ID" and Order data has "Order ID" which is probably the same. They do however have different formats. Example of Order ID in delivery data is "YR-11001517,0" while for order data is "11001517".
2. Customer and Order Data have {'Customer ID'} in common and it is of unique ID type. 
3. Customer and Deliveries Data do not have columns in common

In [32]:
# Rename columns to match columns in other dataframes
del_df = del_df.rename(columns={"Order_ID": "Order ID"})

In [33]:
# Match pattern of "Order ID" in orders and deliveries data
pat=r'YR-(\d+),0'
del_df["Order ID"] = del_df["Order ID"].str.replace(pat, r'\1', regex=True)


In [34]:
# Type cast to string all common columns
del_df["Order ID"]=del_df["Order ID"].astype(str)
ord_df["Order ID"]=ord_df["Order ID"].astype(str)
cust_df["Customer ID"]=cust_df["Customer ID"].astype(str)
ord_df["Customer ID"]=ord_df["Customer ID"].astype(str)



In [35]:
# Check 
ord_ord_set = set(ord_df["Order ID"])
del_ord_set = set(del_df["Order ID"])
ord_cust_set = set(ord_df["Customer ID"])
cust_cust_set = set(cust_df["Customer ID"])


### How the common columns relate with each other

In [38]:
print("How many Order ID are in both Orders and Deliveries data Kenya? "+str(len(del_ord_set&ord_ord_set)))
print("Are all Order IDs in deliveries data in orders data? "+ str(del_ord_set.issubset(ord_ord_set)))
print("Are all Order IDs in orders data in deliveries data? "+ str(ord_ord_set.issubset(del_ord_set)))
print("How many Order IDs in orders data are not in deliveries data? "+ str(len(del_ord_set.difference(ord_ord_set))))
print("How many Order IDs in deliveries data are not in orders data? "+ str(len(ord_ord_set.difference(del_ord_set))))


How many Order ID are in both Orders and Deliveries data Kenya? 4391
Are all Order IDs in deliveries data in orders data? False
Are all Order IDs in orders data in deliveries data? False
How many Order IDs in orders data are not in deliveries data? 21640
How many Order IDs in deliveries data are not in orders data? 4


**NB: The orders are only for this year while most of the deliveries are for last year, this is why most of the Order IDs in deliveries data are not in orders data**

In [39]:
# Check if all Orders ID in deliveries are in Orders Data
print("How many Customer ID are in both Orders and Customer data? "+str(len(ord_cust_set&cust_cust_set)))
print("Are all Customer IDs in orders data in customer data? "+ str(ord_cust_set.issubset(cust_cust_set)))
print("Are all Customer IDs in customer data in order data? "+ str(cust_cust_set.issubset(ord_cust_set)))
print("How many Customer IDs in orders data are not in customer data? "+ str(len(ord_cust_set.difference(cust_cust_set))))
print("How many Customer IDs in customer data are not in orders data? "+ str(len(cust_cust_set.difference(ord_cust_set))))

How many Customer ID are in both Orders and Customer data? 702
Are all Customer IDs in orders data in customer data? True
Are all Customer IDs in customer data in order data? False
How many Customer IDs in orders data are not in customer data? 0
How many Customer IDs in customer data are not in orders data? 4570


**NB: All Customer IDs in orders are data are in customer data, this makes sense cause every orders should be mapped to a customer**

### **Merge the data set on common colums**

In [43]:
# Initialize the common columns to merge on
cust_cols = ['Customer ID','country_code']
ord_cols = ['Order ID','country_code','Discount','Tip']

# Merge orders data and customer data on 'Customer ID','country_code'
ord_cust_df = pd.merge(cust_df, ord_df, left_on=cust_cols, right_on = cust_cols, how="outer")

# Merge the new merged data frame and deliveries data on 'Order ID','country_code','Discount','Tip'
df = pd.merge(ord_cust_df, del_df, left_on=ord_cols, right_on = ord_cols, how="outer")
df.tail()

,Customer ID,Last Used Platform,Is Blocked,Created At,Language,Outstanding Amount,Loyalty Points,Number of Employees,Upload restuarant location,country_code,...,Task_Details_AMOUNT,Special_Instructions,Delivery_Charges,Subtotal,Payment_Type,Task_Category,Earning,Pricing,Unnamed: 34,Unnamed: 35
104590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,234,...,₦ 10000.00,-,₦ 0.00,10000,CASH,-,-,-,NaN,NaN
104591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,234,...,₦ 17000.00,-,₦ 0.00,17000,CASH,-,-,-,NaN,NaN
104592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,234,...,₦ 17000.00,-,-,17000,CASH,-,-,-,NaN,NaN
104593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,234,...,₦ 10000.00,-,₦ 0.00,10000,CASH,-,-,-,NaN,NaN
104594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,234,...,₦ 10000.00,-,-,10000,CASH,-,-,-,NaN,NaN
